In [66]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from subprocess import check_output
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn import grid_search
from random import seed
from random import randrange

from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

import xgboost
import pickle
%matplotlib inline

### Reading the raw data into a dataframe

In [2]:
#decision_date and case_recieved_date are read as dates
df = pd.read_csv('us_perm_visas.csv', low_memory = False, parse_dates=['decision_date', 'case_received_date'],\
                 index_col = ['case_no'])

In [3]:
#Removing all withdrawn applications
df = df[df.case_status != 'Withdrawn']

#Combining certified-expired and certified applications
df.loc[df.case_status == 'Certified-Expired', 'case_status'] = 'Certified'

#Now only two labels, Certified and Denied, which are stored in the column case_status

In [4]:
#Convering the date to just the year
df['year'] = df['case_received_date'].dt.year

In [5]:
#Dropping all empty columns
df = df.dropna(axis=1, how='all');

#Dropping all empty rows
df = df.dropna(axis=0, how='all');

In [6]:
#Dropping all rows with any missing values
df_nona = df.dropna(axis=0, how='any');

df_nona.info()

#This shows that dropping rows with empty values is not an option

### Visualisation of the Unprocessed Data

In [7]:
pd.set_option('display.max_colwidth', -1)

In [8]:
i = 0
count_nonnan = [];
ct = [];

for col in df.columns:
    i+=1
    count_nonnan.append(df[col].count())
    ct.append(i)

In [9]:
plt.scatter(count_nonnan, ct)
plt.ylabel('Feature Number', fontsize=16)
plt.xlabel('Number of non-empty entries', fontsize=16)

#This is a scatterplot to show the number of non-empty values per feature

In [10]:
#Total Number of data points of each class
df['case_status'].value_counts().plot(kind = 'bar')

In [11]:
print('There are', df['country_of_citizenship'].value_counts().count(), \
      'different countries of citizenship in this dataset')
df['country_of_citizenship'].value_counts().head(10).plot(kind = 'barh',color = 'maroon', alpha=0.7)

#Graph showing the number of applicants from the countried with the 10 highest number of applicants and the total number of countries

In [12]:
#Graph showing the number of Certified and Denied visa applications for the top 10 countries by number of visa applicants

sns.set_style("whitegrid")
fig, ax = plt.subplots()
fig.set_size_inches(11.7, 8.27)
sns.countplot(ax = ax, x='country_of_citizenship', data = df, hue='case_status', \
              order=df.country_of_citizenship.value_counts().iloc[:10].index,alpha = 0.5)#, palette=pkmn_type_colors)

### Feature Selection

In [13]:
#Displaying percentage of non-null values for each feature

i = 0;
for col in df.columns:
    i = i+1;
    print (i-1, col, 100*df[col].count()/len(df['case_status']))

In [14]:
#Finding relation between case_received_date and decision_date
df[['case_received_date', 'decision_date']]

#Since they are almost the same, and the decision_date is 100% filled, we can drop case_recieved_date

In [15]:
#Indices of selected features
a = [2,3,7,8,9,11,15,17,18,22,23,77,90,96,98,101];

#These features were selected based on the number of empty values and relevance to the visa application decision

df2 = df.iloc[:,a];
df2 = df2.dropna(thresh=0.5*len(df), axis=1);
df2.info()

#selected features are displayed

### Selected Features

In [16]:
import matplotlib.colors as mcolors

def make_colormap(seq):
    """Return a LinearSegmentedColormap
    seq: a sequence of floats and RGB-tuples. The floats should be increasing
    and in the interval (0,1).
    """
    seq = [(None,) * 3, 0.0] + list(seq) + [1.0, (None,) * 3]
    cdict = {'red': [], 'green': [], 'blue': []}
    for i, item in enumerate(seq):
        if isinstance(item, float):
            r1, g1, b1 = seq[i - 1]
            r2, g2, b2 = seq[i + 1]
            cdict['red'].append([item, r1, r2])
            cdict['green'].append([item, g1, g2])
            cdict['blue'].append([item, b1, b2])

    return mcolors.LinearSegmentedColormap('CustomMap', cdict)

c = mcolors.ColorConverter().to_rgb
rvb = make_colormap(
[c('red'), 0.125, c('red'), c('orange'), 0.25, c('orange'),c('green'),0.5, c('green'),0.7, c('green'), c('blue'), 0.75, c('blue')])

N = 60
x = np.arange(N).astype(float)
#y = np.random.uniform(0, 5, size=(N,))

#plt.bar(x,y, color=rvb(x/N),alpha = 0.5)
#plt.show()


In [17]:

#Displaying percentage of non-null values for each selected feature
objects = []
counts = []
i = 0;
for col in df2.columns:
    i = i+1;
    objects.append(col)
    counts.append(100*df2[col].count()/len(df2['case_status']))
    
y_pos = np.arange(len(objects))
plt.figure(figsize=(10, 10))  # width:20, height:3
plt.barh(y_pos, counts, align='edge', alpha=0.5, color = rvb(x/30))
plt.yticks(y_pos, objects)
plt.xlabel('Percentage Filled')
plt.title('Selected Features')
 
plt.show()

#Features with more than 90% non-empty values can be filled in using the mean/mode
#Features with lesser than 90% non-empty values, other methods are used

### Data Cleaning

In [18]:
#Assigning Labels to Case Status
df2.loc[df.case_status == 'Certified', 'case_status'] = 1
df2.loc[df.case_status == 'Denied', 'case_status'] = 0

#Calculating pay based on unit of pay and wage
di = {"Year": 1, "yr": 1, "Hour": 2080, "hr": 2080, "Week": 52, "wk": 52, "Bi-Weekly": 26,"bi": 26, "Month": 12, "mth": 12}
df2['pw_unit_of_pay_9089'].replace(di, inplace=True)

#Changing datatype of pay to float
df2['pw_amount_9089'] = df2['pw_amount_9089'].str.replace(',', '')
df2[['pw_amount_9089', 'pw_unit_of_pay_9089']] = df2[['pw_amount_9089', 'pw_unit_of_pay_9089']].astype(float)
df2['pw_amount_9089_new'] = df2['pw_amount_9089']*df2['pw_unit_of_pay_9089']
df2['pw_amount_9089_new']=df2['pw_amount_9089_new'].fillna((df2['pw_amount_9089_new'].mean()))

#Dropping redundant pay related features
df2 = df2.drop('pw_amount_9089', 1)
df2 = df2.drop('pw_unit_of_pay_9089', 1)

In [19]:
#Extracting year from decision date and converting to an integer value
df2['decision_date'] = df2['decision_date'].dt.year
df2['decision_date'] = df2['decision_date'].astype(int)
df2['decision_date']=df2['decision_date'].fillna((df2['decision_date'].mean()))

In [20]:
#Since there are too many unknown values, it is treated as a new value
df2['agent_state'].fillna('Unknown');

#Mapping from state name to short-form
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'Northern Mariana Islands':'MP', 
    'Palau': 'PW', 
    'Puerto Rico': 'PR', 
    'Virgin Islands': 'VI', 
    'District of Columbia': 'DC'
}

#Capitalizing Keys
us_state_abbrev = {k.upper(): v for k, v in us_state_abbrev.items()}
df2['agent_state'].replace(us_state_abbrev, inplace=True)
df2.agent_state = df2.agent_state.astype(str)


In [21]:
#Similarly for Employer State
df2['employer_state'] = df2['employer_state'].fillna(df2['employer_state'].mode()[0]);

#Capitalizing Keys
us_state_abbrev = {k.upper(): v for k, v in us_state_abbrev.items()}
df2['employer_state'].replace(us_state_abbrev, inplace=True)
df2.employer_state = df2.employer_state.astype(str)


#Similarly for Job info work State
df2['job_info_work_state'] = df2['job_info_work_state'].fillna(df2['job_info_work_state'].mode()[0]);

#Capitalizing Keys
us_state_abbrev = {k.upper(): v for k, v in us_state_abbrev.items()}
df2['job_info_work_state'].replace(us_state_abbrev, inplace=True)
df2.job_info_work_state = df2.job_info_work_state.astype(str)



In [22]:
#df2['employer_postal_code'] = df2['employer_postal_code'].str.replace('.','')
#df2['employer_postal_code'] = df2['employer_postal_code'].str.replace('-','')
#df2['employer_postal_code'] = df2['employer_postal_code'].astype(str).str[0:4]
#empty_emp_index = df2['employer_postal_code'] == "nan"
#df2['employer_postal_code'][empty_emp_index] = '0000';
#not_num_values = df2['employer_postal_code'].str.contains('[a-zA-Z]')
#df2['employer_postal_code'][not_num_values] = '0000';
#df2['employer_postal_code'] = df2['employer_postal_code'].astype(int)
df2['pw_soc_code'] = df2['pw_soc_code'].str.replace('.','')
df2['pw_soc_code'] = df2['pw_soc_code'].str.replace('-','')
df2['pw_soc_code'] = df2['pw_soc_code'].astype(str).str[0:6]
df2['pw_soc_code'].value_counts()
empty_soc_index = df2['pw_soc_code'] == "nan"
df2['pw_soc_code'][empty_soc_index] = df2['pw_soc_code'].mode()[0];

empty_soc_index = df2['pw_soc_code'] == "None"
df2['pw_soc_code'][empty_soc_index] = df2['pw_soc_code'].mode()[0];

In [23]:
df2['pw_soc_code'] = df2['pw_soc_code'].astype(float)

df2['pw_soc_code'] = df2['pw_soc_code'].astype(int)

df2['case_status'] = df2['case_status'].astype(int)

In [24]:
#Displaying percentage of non-null values for each selected feature

i = 0;
for col in df2.columns:
    i = i+1;
    print (i-1, col, 100*df2[col].count()/len(df2['case_status']))

In [25]:
#Replace the values with the mode or mean
df2['class_of_admission']=df2['class_of_admission'].fillna((df2['class_of_admission'].mode()[0]))

df2['country_of_citizenship']=df2['country_of_citizenship'].fillna((df2['country_of_citizenship'].mode()[0]))

#df2['employer_city']=df2['employer_city'].fillna((df2['employer_city'].mode()[0]))

df2['employer_name']=df2['employer_name'].fillna((df2['employer_name'].mode()[0]))
df2['employer_name']=df2['employer_name'].astype(str).str.upper()


#df2['job_info_work_city']=df2['job_info_work_city'].astype(str).str.upper()
#df2['job_info_work_city']=df2['job_info_work_city'].fillna((df2['job_info_work_city'].mode()[0].upper()))


df2['pw_source_name_9089']=df2['pw_source_name_9089'].fillna((df2['pw_source_name_9089'].mode()[0]))


df2['pw_soc_code'] = df2['pw_soc_code'].astype(str)
df2['pw_soc_code'] = df2['pw_soc_code'].str.replace('-', '')
df2['pw_soc_code'] = df2['pw_soc_code'].str[0:6]

df2['pw_soc_code']=df2['pw_soc_code'].fillna((df2['pw_soc_code'].mode()[0]))

df2['employer_yr_estab']=df2['employer_yr_estab'].fillna(1700.0)
df2['employer_yr_estab'] = df2['employer_yr_estab'].astype(int)
df2['employer_yr_estab'].value_counts()
df2.loc[df['employer_yr_estab'] < 1700, 'employer_yr_estab'] = 1700

df2['employer_country']=df2['employer_country'].fillna((df2['employer_country'].mode()[0]))

df2['employer_num_employees']=df2['employer_num_employees'].fillna((df2['employer_num_employees'].mean()))


In [26]:
#Displaying percentage of non-null values for each selected feature

i = 0;
for col in df2.columns:
    i = i+1;
    print (i-1, col, 100*df2[col].count()/len(df2['case_status']))

### Visualizing the filled features

In [27]:
df2['pw_amount_9089_new'].head(25).plot(kind='hist',color = 'purple')


In [28]:
df2['employer_state'].value_counts().head(15).plot(kind='barh',color= rvb(x/20),alpha = 0.8)

In [29]:
df2['class_of_admission'].value_counts().head(15).plot(kind='barh',alpha = 0.9, color ='pink')

### Converting all data to Categories

In [30]:
converter_dict = {}
#Categorising agent firm name
df2.agent_firm_name = df2.agent_firm_name.astype(str)

le_agent_state = preprocessing.LabelEncoder()
le_agent_state.fit(df2['agent_state'])
df2['agent_state'] = le_agent_state.transform(df2['agent_state'])
converter_dict["agent_state"] = le_agent_state

le_employer_state = preprocessing.LabelEncoder()
le_employer_state.fit(df2['employer_state'])
df2['employer_state'] = le_employer_state.transform(df2['employer_state'])
converter_dict["employer_state"] = le_employer_state

le_agent_firm_name = preprocessing.LabelEncoder()
le_agent_firm_name.fit(df2['agent_firm_name'])
df2['agent_firm_name'] = le_agent_firm_name.transform(df2['agent_firm_name'])
converter_dict["agent_firm_name"] = le_agent_firm_name

le_class_of_admission = preprocessing.LabelEncoder()
le_class_of_admission.fit(df2['class_of_admission'])
df2['class_of_admission'] = le_class_of_admission.transform(df2['class_of_admission'])
converter_dict["class_of_admission"] = le_class_of_admission

le_country_of_citizenship = preprocessing.LabelEncoder()
le_country_of_citizenship.fit(df2['country_of_citizenship'])
df2['country_of_citizenship'] = le_country_of_citizenship.transform(df2['country_of_citizenship'])
converter_dict["country_of_citizenship"] = le_country_of_citizenship

#employer_city
#le_employer_city = preprocessing.LabelEncoder()
#le_employer_city.fit(df2['employer_city'])
#df2['employer_city'] = le_employer_city.transform(df2['employer_city'])
#converter_dict["employer_city"] = le_employer_city

#employer_country
le_employer_country = preprocessing.LabelEncoder()
le_employer_country.fit(df2['employer_country'])
df2['employer_country'] = le_employer_country.transform(df2['employer_country'])
converter_dict["employer_country"] = le_employer_country

#employer_name
le_employer_name = preprocessing.LabelEncoder()
le_employer_name.fit(df2['employer_name'])
df2['employer_name'] = le_employer_name.transform(df2['employer_name'])
converter_dict["employer_name"] = le_employer_name

#job_info_work_ciity
#le_job_info_work_city = preprocessing.LabelEncoder()
#le_job_info_work_city.fit(df2['job_info_work_city'])
#df2['job_info_work_city'] = le_job_info_work_city.transform(df2['job_info_work_city'])
#converter_dict["job_info_work_city"] = le_job_info_work_city

#job_info_work_state
le_job_info_work_state = preprocessing.LabelEncoder()
le_job_info_work_state.fit(df2['job_info_work_state'])
df2['job_info_work_state'] = le_job_info_work_state.transform(df2['job_info_work_state'])
converter_dict["job_info_work_state"] = le_job_info_work_state

#pw_source_name_9089
le_pw_source_name_9089 = preprocessing.LabelEncoder()
le_pw_source_name_9089.fit(df2['pw_source_name_9089'])
df2['pw_source_name_9089'] = le_pw_source_name_9089.transform(df2['pw_source_name_9089'])
converter_dict["pw_source_name_9089"] = le_pw_source_name_9089

print converter_dict

f = open("saved_mapping","wb")
pickle.dump(converter_dict, f)
f.close()

## Application Decision Prediction

### Random Forest Classifier

In [31]:
X = df2.loc[:, df2.columns != 'case_status']
Y = df2.case_status

f = open("data_d2","wb")
pickle.dump(df2, f)
f.close()

df2.iloc[0]

In [32]:
X.shape

In [33]:
Y.shape

In [34]:
#X_try = [[1,1,3,2],[2,3,4,3]]
#Y_try = [1,0]

In [35]:
#estimator = SVR(kernel="linear")
#selector = RFE(estimator, 1, step=2,verbose=True)
#selector = selector.fit(X_try, Y_try)
#selector.support_
#selector.ranking_

In [36]:
#estimator = SVR(kernel="linear")
#selector = RFE(estimator, 5, step=16,verbose=True)
#selector = selector.fit(X[0:1000], Y[0:1000])
#selector.support_
#selector.ranking_

In [37]:
skf = StratifiedKFold(n_splits = 5)

In [38]:
#Parameters for random forest. To perform hyper parameter optimisation a list of multiple elements can be entered
#and the optimal value in that list will be picked using grid search
def parameter_set_random_forest(n_estimators = [10], criterion = ['gini'], max_depth = [None],\
                                min_samples_split = [2], min_samples_leaf = [1], min_weight_fraction_leaf = [0.0],\
                                max_features = ['auto'], max_leaf_nodes = [None], bootstrap = [True],\
                                oob_score = [False], random_state = [None], verbose = [0],warm_start = [False],\
                                class_weight = [None]):
    
    parameters_random_forest = {}
    parameters_random_forest['criterion'] = criterion
    parameters_random_forest['n_estimators'] = n_estimators
    parameters_random_forest['max_depth'] = max_depth
    parameters_random_forest['min_samples_split'] = min_samples_split
    parameters_random_forest['min_samples_leaf'] = min_samples_leaf
    parameters_random_forest['min_weight_fraction_leaf'] = min_weight_fraction_leaf
    parameters_random_forest['max_features'] = max_features
    parameters_random_forest['random_state'] = random_state
    parameters_random_forest['max_leaf_nodes'] = max_leaf_nodes
    parameters_random_forest['class_weight'] = class_weight
    parameters_random_forest['bootstrap'] = bootstrap
    parameters_random_forest['oob_score'] = oob_score
    parameters_random_forest['warm_start'] = warm_start
    
    return parameters_random_forest

In [39]:
models = []
for train_index, test_index in skf.split(X, Y):
    x_train = X.iloc[train_index,:]
    x_test = X.iloc[test_index,:]
    y_train = Y.iloc[train_index]
    y_test = Y.iloc[test_index]
    print("TRAIN:", x_train.shape, "TEST:", x_test.shape)
    random_forest_model = RandomForestClassifier()
    parameters_random_forest = parameter_set_random_forest(n_estimators=[10,20,30,40],max_depth=[35,50,75,100,None])
    model_gs = grid_search.GridSearchCV(random_forest_model, parameters_random_forest, scoring = 'roc_auc')
    model_gs.fit(x_train,y_train)
    predictions = model_gs.predict_proba(x_test)[:,1]
    train_acc = roc_auc_score(y_train, model_gs.predict_proba(x_train)[:,1])
    test_acc = roc_auc_score(y_test, predictions)
    print ("Train Accuracy :: ", roc_auc_score(y_train, model_gs.predict_proba(x_train)[:,1]))
    print ("Test Accuracy  :: ", roc_auc_score(y_test, predictions))
    models.append((model_gs,x_train,y_train,x_test,y_test,train_acc,test_acc))

In [40]:
model_out = open("saved_models_2","wb")
pickle.dump(models, model_out)
model_out.close()

In [41]:
model_in = open("saved_models_2","rb")
models_load  = pickle.load(model_in)
model_in.close()

In [44]:
df2.head()

In [43]:
models_load

### Decision Tree Classifier

In [ ]:
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    #print(len(dataset_copy), n_folds)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            #print(len(dataset_copy))
            #if(index == 0):
             #   break;
            #index = index - 1
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)    
        else:
            right.append(row)
    return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

# Select the best split point for a dataset
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

# Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

seed(1)
# load and prepare data
dataset = df2.loc[:, df2.columns != 'case_status']
dataset['case_status'] = df2.loc[:, df2.columns == 'case_status']
dataset = dataset.values.tolist()[1:6000]
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

In [46]:
Y.value_counts()